# Comparison of Solutions for Markov Decision Processes

In this project, we will be comparing a variety of classic dynamic programming and linear programming methods. From the dynamic programming realm, the solution methods are:
-  Value Iteration
-  Policy Iteration

and for linear programming methods:
-  First order methods
-  Interior point methods
-  Simplex methods

## Introduction

Markov Decision Processes (MDPs) provide a mathematical formulation for stochastic decision making

In [2]:
import numpy as np
import numpy.linalg as la
import numpy.random as rn
import matplotlib.pyplot as plt

Below is an example of a candidate Grid World that will be solved using the above methods.

## Generating the MDP

## Value Iteration

## Policy Iteration

## Simplex Method

## First Order Methods

### Gradient Ascent

### Accelerated Gradient Ascent

## Interior Point Methods